In [3]:
"""
The GUI's purpose is to streamline the workflow for medical professionals and researchers by providing an easy-to-use interface for managing and 
processing medical imaging data. 

The script reflects a well-thought-out structure that separates concerns into different tabs and methods, making it maintainable and scalable. 
It seems ready to assist users in the critical tasks of image classification, annotation, and preparation for machine learning model training, 
which are crucial steps in developing AI tools for medical diagnostics.
"""
# Import required libraries
import os
import cv2
import json
import shutil
import random
import labelme   
import subprocess
import numpy as np
import tkinter as tk
from shutil import copyfile
from PIL import Image, ImageTk, ImageDraw
from tkinter import ttk, filedialog, messagebox, simpledialog, PhotoImage

# Main class for the Medical Image GUI application
class MedicalGUI:
    def __init__(self, root):
        # Initialize the main application window with a title and fixed size
        self.root = root
        self.root.title("Glaucoma GUI")
        self.root.geometry("800x600")

        # Initialize lists to store paths of images for classification and labeling
        self.classifier_image_paths = []  # Stores file paths for the classifier
        self.classifier_current_image_index = 0  # Index of the currently displayed image in the classifier
        self.label_data = []  # Stores the label data (annotations) for the labeling tab
        self.labeling_image_path = []        
        self.labeling_image_paths = []  # Stores file paths for the labeling
        self.labeling_current_image_index = 0  # Index of the currently displayed image in the labeling

        # Initialize variables for the training tab
        self.preprocessing_input_folder = ""  # Path to the input folder selected for preprocessing
        self.preprocessing_method = tk.StringVar()  # Selected preprocessing method (e.g., color space conversion)
        self.target_folder = None  # Target folder for saving processed images
        self.data_yaml_path = None  # Path to the YAML file with dataset configuration
        self.hyp_yaml_path = None  # Path to the YAML file with hyperparameters configuration
        self.cfg_yaml_path = None  # Path to the YAML file with model configuration
        self.weights_path = None  # Path to the weights file for model training
        
        # Path to the default image displayed when no images are available
        self.default_image_path = "Imagenes/Caratula.jpg"

        # Create the main interface tabs
        self.create_notebook()
        
    def create_notebook(self):
        # Create a tabbed interface and add tabs for Classifier, Labeling, and Training        
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(expand=True, fill=tk.BOTH)
        self.create_classifier_frame()
        self.create_labeling_frame()
        self.create_training_frame()

    # The following methods define the functionality for each tab in the GUI such as loading images,
    # navigating between images, applying image processing methods, and handling user inputs for
    # image classification, labeling, and preparing data for model training.
    # These methods use tkinter widgets such as buttons, sliders, and canvas to create an interactive interface.
    # The image processing methods make use of OpenCV for reading and manipulating images and LabelMe for annotations.
    # The application is structured to allow users to easily navigate through their image dataset, perform necessary preprocessing,
    # and prepare the data for training a machine learning model for medical diagnosis.
            
################################################################################################################################

    #Step 1: Visual classifier frame
    
    def create_classifier_frame(self):
        self.classifier_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.classifier_frame, text="Classifier")
            
        # Widgets and logic for the classifier frame

        # Create a frame for loading folders buttons
        self.classifier_folders_frame = tk.Frame(self.classifier_frame)
        self.classifier_folders_frame.pack()
        
        # Create buttons for loading input, suspect, and non-suspect folders. 
        # Methods: to_classify_folder, load_non_suspect_folder, load_suspect_folder
        
        # Create a button for input folder
        self.to_classify_button = tk.Button(self.classifier_folders_frame, text="Load folder to classify", command=self.to_classify_folder)
        self.to_classify_button.pack(side=tk.LEFT)  
        # Create a button for non-suspect folder
        self.non_suspect_button = tk.Button(self.classifier_folders_frame, text="Load folder for non-suspect images", command=self.load_non_suspect_folder)
        self.non_suspect_button.pack(side=tk.RIGHT)
        # Create a button for suspect folder
        self.suspect_button = tk.Button(self.classifier_folders_frame, text="Load folder for suspect images", command=self.load_suspect_folder)
        self.suspect_button.pack(side=tk.RIGHT)

        # Create a frame for no img text
        self.classifier_no_img_text_frame = tk.Frame(self.classifier_frame)
        self.classifier_no_img_text_frame.pack()
        # Create a no img text
        self.classifier_image_name_label = tk.Label(self.classifier_no_img_text_frame, text="No images available in the input folder")
        self.classifier_image_name_label.pack()    
                    
        # Create a frame for switch, Gamma label and slider and reset buttons (Gamma correction)
        self.gamma_frame = tk.Frame(self.classifier_frame)
        self.gamma_frame.pack()
        
        # Create buttons for switch RGB-BN, Gamma label and slider, Reset gamma.
        # Methods: toggle_RGB, update_gamma, reset_gamma
        
        # Create a button for RGB-BN
        self.RGB_button = tk.Checkbutton(self.gamma_frame, text="Gray", command=self.toggle_RGB, state=tk.DISABLED)
        self.RGB_button.pack(side=tk.LEFT)
        self.RGB_enabled = True
        # Create Gamma label
        self.gamma_label = tk.Label(self.gamma_frame, text="Gamma:")
        self.gamma_label.pack(side=tk.RIGHT)
        # Create Gamma slider
        self.gamma_slider = tk.Scale(self.gamma_frame, from_=0.1, to=5.0, resolution=0.01, orient=tk.HORIZONTAL, command=self.update_gamma)
        self.gamma_slider.set(1.0)
        self.gamma_slider.pack(side=tk.RIGHT)
        # Create Gamma reset button (Gamma: 1.0)
        self.gamma_reset_button = tk.Button(self.gamma_frame, text="Initial value", command=self.reset_gamma)
        self.gamma_reset_button.pack(side=tk.RIGHT)  
        
        # Create a frame for the "Accept" and "Reject" buttons
        self.decisions_frame = tk.Frame(self.classifier_frame)
        self.decisions_frame.pack()
        
        # Create buttons for Accept and Reject
        # Methods: accept_image, reject_image, classifier_load_previous_image, classifier_load_next_image, classifier_update_image
        
        # Create a button for Accept
        self.accept_button = tk.Button(self.decisions_frame, text="Accept", command=self.accept_image, state=tk.DISABLED)
        self.accept_button.pack(side=tk.LEFT)
        # Create a button for Reject
        self.reject_button = tk.Button(self.decisions_frame, text="Decline", command=self.reject_image, state=tk.DISABLED)
        self.reject_button.pack(side=tk.RIGHT)
        
        # Create frames for the "Previous" and "Next" buttons and a classifier_canvas to display the image
        # Load arrow images

        original_left_arrow_image = PhotoImage(file="Imagenes/left.png")
        original_right_arrow_image = PhotoImage(file="Imagenes/right.png")

        # Define the scale factor (e.g., 2 for half the size)
        scale_factor = 8

        self.left_arrow_image = original_left_arrow_image.subsample(scale_factor, scale_factor)
        self.right_arrow_image = original_right_arrow_image.subsample(scale_factor, scale_factor)
        
        
        self.classifier_previous_button_frame = tk.Frame(self.classifier_frame)
        self.classifier_previous_button_frame.pack(side=tk.LEFT, fill=tk.Y)
        self.classifier_next_button_frame = tk.Frame(self.classifier_frame)
        self.classifier_next_button_frame.pack(side=tk.RIGHT, fill=tk.Y)
        self.classifier_canvas = tk.Canvas(self.classifier_frame, width=800, height=600, bg="black")
        self.classifier_canvas.pack(fill=tk.X, expand=True)
        
        # Create buttons for load the previous and next image, and update canvas.
        # Methods: classifier_load_previous_image, classifier_load_next_image, classifier_update_image

        # Create a button to load the previous image
        self.classifier_previous_button = ttk.Button(self.classifier_previous_button_frame, image=self.left_arrow_image, command=self.classifier_load_previous_image, state=tk.DISABLED)
        self.classifier_previous_button.grid(row=1, column=0, padx=5, pady=150, sticky="ew")
        
        # Create a button to load the next image
        self.classifier_next_button = ttk.Button(self.classifier_next_button_frame, image=self.right_arrow_image, command=self.classifier_load_next_image, state=tk.DISABLED)
        self.classifier_next_button.grid(row=1, column=2, padx=5, pady=150, sticky="ew")

        
        # Bind the <Configure> event to update the image when the classifier_canvas size changes
        self.classifier_canvas.bind("<Configure>", self.classifier_update_image)
              
    # Add the new methods to the MedicalGUI class
    
    # classifier_folders_frame
    # Methods: classifier_load_image, classifier_load_default_image, classifier_update_buttons

    def to_classify_folder(self):
        to_classify_path = filedialog.askdirectory(title="Select input images folder")
        if to_classify_path:
            image_extensions = ['.jpg', '.jpeg', '.png', '.tif', '.TIF', '.PNG', '.JPG', '.JPEG']
            self.classifier_image_paths = [os.path.join(to_classify_path, file) for file in os.listdir(to_classify_path) if any(file.endswith(ext) for ext in image_extensions)]
            if len(self.classifier_image_paths) > 0:
                print("The input folder contains " + str(len(self.classifier_image_paths)) + " images")
                self.classifier_current_image_index = 0
                self.classifier_load_image(self.classifier_image_paths[self.classifier_current_image_index])
            else:
                print("Input folder does not contain images")
                self.classifier_load_default_image()
        self.classifier_update_buttons()
        self.RGB_button.config(state=tk.NORMAL)

    def load_non_suspect_folder(self):
        non_suspect_path = filedialog.askdirectory(title="Select the folder for images without suspected RNFLd.")
        if non_suspect_path:
            self.non_suspect_folder_path = non_suspect_path    
        self.classifier_update_buttons()

    def load_suspect_folder(self):
        suspect_path = filedialog.askdirectory(title="Select the folder for images with suspected RNFLd.")
        if suspect_path:
            self.suspect_folder_path = suspect_path
        self.classifier_update_buttons()        
           
    # gamma_frame
    # Methods: classifier_load_image, classifier_update_buttons
    
    def toggle_RGB(self):
        self.RGB_enabled = not self.RGB_enabled
        self.classifier_load_image(self.classifier_image_paths[self.classifier_current_image_index])        
        
    def update_gamma(self, value):
        if not self.classifier_image_paths:
            return # si la lista classifier_image_paths esta vacia, no hacer nada
        # Obtener la ruta de la imagen actual (image_path)
        clasiffier_image_path = self.classifier_image_paths[self.classifier_current_image_index]
        # Cargar la imagen con la corrección gamma actualizada
        self.classifier_load_image(clasiffier_image_path)
        self.classifier_update_buttons()
        
    def reset_gamma(self):
        self.gamma_slider.set(1.0)
        self.classifier_update_buttons()      
                      
    # decisions_frame
    # Methods: classifier_load_image, classifier_load_default_image, classifier_update_buttons
    
    def accept_image(self):
        classifier_filename = os.path.basename(self.classifier_image_paths[self.classifier_current_image_index])
        shutil.move(self.classifier_image_paths[self.classifier_current_image_index], os.path.join(self.suspect_folder_path, classifier_filename))
        self.classifier_image_paths.pop(self.classifier_current_image_index)
        if len(self.classifier_image_paths) > 0:
            self.classifier_current_image_index %= len(self.classifier_image_paths)
            clasiffier_image_path = self.classifier_image_paths[self.classifier_current_image_index]
            self.classifier_load_image(clasiffier_image_path)
        else:
            self.classifier_load_default_image()
        self.classifier_update_buttons()

    def reject_image(self):
        # Move the image to the non-suspect folder
        classifier_filename = os.path.basename(self.classifier_image_paths[self.classifier_current_image_index])
        shutil.move(self.classifier_image_paths[self.classifier_current_image_index], os.path.join(self.non_suspect_folder_path, classifier_filename))
        self.classifier_image_paths.pop(self.classifier_current_image_index)
        if len(self.classifier_image_paths) > 0:
            self.classifier_current_image_index %= len(self.classifier_image_paths)
            clasiffier_image_path = self.classifier_image_paths[self.classifier_current_image_index]
            self.classifier_load_image(clasiffier_image_path)
        else:
            self.classifier_load_default_image()
        self.classifier_update_buttons()       
        
    # classifier_previous_button_frame, classifier_next_button_frame, classifier_canvas
    # Methods: classifier_load_image, classifier_load_default_image
    
    def classifier_load_previous_image(self):
        if len(self.classifier_image_paths) > 0:
            self.classifier_current_image_index = (self.classifier_current_image_index - 1) % len(self.classifier_image_paths)
            clasiffier_image_path = self.classifier_image_paths[self.classifier_current_image_index]
            self.classifier_load_image(clasiffier_image_path)
        else:
            # Si no hay más imágenes, cargar la imagen por defecto "Caratula.jpg"
            self.classifier_current_image_index = 0
            self.classifier_load_default_image()
            print("No images available in the input folder")
        
    def classifier_load_next_image(self):
        if len(self.classifier_image_paths) > 0:
            self.classifier_current_image_index = (self.classifier_current_image_index + 1) % len(self.classifier_image_paths)
            clasiffier_image_path = self.classifier_image_paths[self.classifier_current_image_index]
            self.classifier_load_image(clasiffier_image_path)
        else:
            # Si no hay más imágenes, cargar la imagen por defecto "Caratula.jpg"
            self.classifier_current_image_index = 0
            self.classifier_load_default_image()
            print("No images available in the input folder")   

    def classifier_update_image(self, event):
        # Update classifier_canvas dimensions
        self.classifier_canvas.config(width=event.width, height=event.height)
    
        # Check if there is a current image to display
        if hasattr(self, 'classifier_current_image_index') and len(self.classifier_image_paths) > 0:
            # Reload the current image with the new classifier_canvas dimensions
            self.classifier_load_image(self.classifier_image_paths[self.classifier_current_image_index])
        else:
            # If there is no current image, load the default image with the new classifier_canvas dimensions
            self.classifier_load_default_image()

    # General 
    # Methods: adjust_gamma
    
    def classifier_load_image(self, classifier_image_path):
        if self.RGB_enabled:
            classifier_image = cv2.imread(classifier_image_path)
            classifier_image = cv2.cvtColor(classifier_image, cv2.COLOR_BGR2RGB)
            classifier_canvas_width = self.classifier_canvas.winfo_width()
            classifier_canvas_height = self.classifier_canvas.winfo_height()
            # Calcular los factores de escala para ajustar el ancho y el alto
            classifier_width_scale_factor = classifier_canvas_width / classifier_image.shape[1]
            classifier_height_scale_factor = classifier_canvas_height / classifier_image.shape[0]
            # Calcular el factor de escala para que la imagen cubra la mayor área posible del lienzo sin deformar su relación de aspecto
            classifier_scale_factor = min(classifier_width_scale_factor, classifier_height_scale_factor)
            # Escalar la imagen manteniendo su relación de aspecto
            classifier_scaled_image = cv2.resize(classifier_image, (0, 0), fx=classifier_scale_factor, fy=classifier_scale_factor)   
            # Obtener el valor actual del control deslizante de gamma
            gamma = self.gamma_slider.get()
            # Aplicar la corrección gamma a la imagen
            gamma_image = self.adjust_gamma(classifier_scaled_image, gamma)
            # Crear una imagen de tipo PIL.ImageTk.PhotoImage a partir de la imagen escalada
            classifier_pil_image = Image.fromarray(gamma_image)        
        else:   
            classifier_image = cv2.imread(classifier_image_path,cv2.IMREAD_GRAYSCALE)
            classifier_canvas_width = self.classifier_canvas.winfo_width()
            classifier_canvas_height = self.classifier_canvas.winfo_height()
            # Calcular los factores de escala para ajustar el ancho y el alto
            classifier_width_scale_factor = classifier_canvas_width / classifier_image.shape[1]
            classifier_height_scale_factor = classifier_canvas_height / classifier_image.shape[0]
            # Calcular el factor de escala para que la imagen cubra la mayor área posible del lienzo sin deformar su relación de aspecto
            classifier_scale_factor = min(classifier_width_scale_factor, classifier_height_scale_factor)
            # Escalar la imagen manteniendo su relación de aspecto
            classifier_scaled_image = cv2.resize(classifier_image, (0, 0), fx=classifier_scale_factor, fy=classifier_scale_factor)   
            # Obtener el valor actual del control deslizante de gamma
            gamma = self.gamma_slider.get()
            # Aplicar la corrección gamma a la imagen
            gamma_image = self.adjust_gamma(classifier_scaled_image, gamma)
            # Crear una imagen de tipo PIL.ImageTk.PhotoImage a partir de la imagen escalada
            classifier_pil_image = Image.fromarray(gamma_image)      
        self.classifier_image = ImageTk.PhotoImage(classifier_pil_image)
        # Display the image on the canvas
        self.classifier_canvas.delete("all")
        self.classifier_canvas.create_image(classifier_canvas_width/2, classifier_canvas_height/2, image=self.classifier_image)
        if len(self.classifier_image_paths) > 0:
            self.classifier_canvas.create_text(100, classifier_canvas_height - 30, text=f"Imagen {self.classifier_current_image_index+1} de {len(self.classifier_image_paths)}", fill="white", font=("Arial", 10))
        self.classifier_image_name_label.config(text=f"Imagen cargada: {os.path.basename(classifier_image_path)}")
        self.classifier_update_buttons()
        
    def classifier_load_default_image(self):
        image = cv2.imread(self.default_image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        classifier_canvas_width = self.classifier_canvas.winfo_width()
        classifier_canvas_height = self.classifier_canvas.winfo_height()
        # Calculate the scale factors for width and height
        classifier_width_scale_factor = classifier_canvas_width / image.shape[1]
        classifier_height_scale_factor = classifier_canvas_height / image.shape[0]
        # Calculate the scale factor to make the image cover the largest possible area of the canvas without distorting its aspect ratio
        classifier_scale_factor = min(classifier_width_scale_factor, classifier_height_scale_factor)
        # Scale the image while maintaining its aspect ratio
        classifier_scaled_image = cv2.resize(image, (0, 0), fx=classifier_scale_factor, fy=classifier_scale_factor)
        classifier_pil_image = Image.fromarray(classifier_scaled_image)
        self.classifier_image = ImageTk.PhotoImage(classifier_pil_image)
        # Display the image on the canvas
        self.classifier_canvas.delete("all")
        self.classifier_canvas.create_image(classifier_canvas_width/2, classifier_canvas_height/2, image=self.classifier_image)         

    def classifier_update_buttons(self):
        classifier_num_images = len(self.classifier_image_paths)
        if classifier_num_images > 0:
            self.classifier_previous_button.config(state=tk.NORMAL)
            self.classifier_next_button.config(state=tk.NORMAL)
            if hasattr(self, 'non_suspect_folder_path'):
                self.reject_button.config(state=tk.NORMAL)
            else:
                self.reject_button.config(state=tk.DISABLED)
            if hasattr(self, 'suspect_folder_path'):
                self.accept_button.config(state=tk.NORMAL)
            else:
                self.accept_button.config(state=tk.DISABLED)
        else:
            self.classifier_previous_button.config(state=tk.DISABLED)
            self.classifier_next_button.config(state=tk.DISABLED)
            self.accept_button.config(state=tk.DISABLED)
            self.reject_button.config(state=tk.DISABLED)
            self.classifier_image_name_label.config(text="No images available in the input folder")

        
    def adjust_gamma(self, classifier_image, gamma):
        # Normalizar los valores de los píxeles
        classifier_image = classifier_image / 255.0
        # Aplicar la corrección gamma
        image_gamma = np.power(classifier_image, gamma)
        # Escalar los valores de los píxeles al rango [0, 255]
        image_gamma = np.uint8(image_gamma * 255)
        return image_gamma   
        
################################################################################################################################

    #Step 2: Implement the labeling frame            
    
    def create_labeling_frame(self):
        self.labeling_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.labeling_frame, text="Labeling")

        # Widgets and logic for the labeling frame
            
        # Create a frame for input folder, standarization (format and shape), text no image and labelme buttons
        self.labeling_folders_frame = tk.Frame(self.labeling_frame)
        self.labeling_folders_frame.pack()
        
        # Create buttons for input folder, standarization (format and shape), labelme and no img text.
        # Methods: labeling_load_image_folder, labeling_standarization_process, open_labelme
        
        # Create a button for input folder
        self.labeling_load_image_folder_button = tk.Button(self.labeling_folders_frame, text="Upload folder to label", command=self.labeling_load_image_folder)
        self.labeling_load_image_folder_button.pack(side=tk.LEFT)
        # Create a button for standarization (format and shape)       
        self.labeling_standarization_button = tk.Button(self.labeling_folders_frame, text="Standardization (Format and form)", command=self.labeling_standarization_process)
        self.labeling_standarization_button.pack(side=tk.LEFT)     
        # Create a button for labelme
        self.open_labelme_button = tk.Button(self.labeling_folders_frame, text="Open Labelme", command=self.open_labelme)
        self.open_labelme_button.pack(side=tk.RIGHT)
        
        # Create a frame for no img text
        self.labeling_no_img_text_frame = tk.Frame(self.labeling_frame)
        self.labeling_no_img_text_frame.pack()
        # Create a no img text
        self.labeling_image_name_label = tk.Label(self.labeling_no_img_text_frame, text="No images available in the input folder")
        self.labeling_image_name_label.pack()   

        # Create frames for the "Previous" and "Next" buttons and a classifier_canvas to display the image
        self.labeling_previous_button_frame = tk.Frame(self.labeling_frame)
        self.labeling_previous_button_frame.pack(side=tk.LEFT, fill=tk.Y)
        self.labeling_next_button_frame = tk.Frame(self.labeling_frame)
        self.labeling_next_button_frame.pack(side=tk.RIGHT, fill=tk.Y)
        self.labeling_canvas = tk.Canvas(self.labeling_frame, width=800, height=600, bg="black")
        self.labeling_canvas.pack(fill=tk.X, expand=True)
        
        # Create buttons for load the previous and next image, and update canvas.
        # Methods: labeling_load_previous_image, labeling_load_next_image, labeling_update_image

        # Create a button to load the previous image
        self.labeling_previous_button = ttk.Button(self.labeling_previous_button_frame, image=self.left_arrow_image, command=self.labeling_load_previous_image, state=tk.DISABLED)
        self.labeling_previous_button.grid(row=1, column=0, padx=5, pady=150, sticky="ew")
            
        # Create a button to load the next image
        self.labeling_next_button = ttk.Button(self.labeling_next_button_frame, image=self.right_arrow_image, command=self.labeling_load_next_image, state=tk.DISABLED)
        self.labeling_next_button.grid(row=1, column=2, padx=5, pady=150, sticky="ew")
            
        # Bind the <Configure> event to update the image when the classifier_canvas size changes
        self.labeling_canvas.bind("<Configure>", self.labeling_update_image)    

            
    # Add the new methods to the MedicalGUI class
    
    # labeling_folders_frame
    # Methods: circle_crop, crop_image_from_gray
    
    def labeling_load_image_folder(self):
        labeling_image_folder = filedialog.askdirectory(title="Upload folder to label")
        if labeling_image_folder:
            self.labeling_image_paths = [os.path.join(labeling_image_folder, file) for file in os.listdir(labeling_image_folder) if file.lower().endswith(('.jpg', '.jpeg', '.png', '.tif', '.TIF', '.PNG', '.JPG', '.JPEG'))]
            if len(self.labeling_image_paths) > 0:
                print("The input folder contains " + str(len(self.labeling_image_paths)) + " images")
                self.labeling_current_image_index = 0
                self.load_image_to_label(self.labeling_image_paths[self.labeling_current_image_index])
            else:
                print("Input folder does not contain images")
                self.labeling_load_default_image()            
        self.labeling_update_buttons()               
    
    def labeling_standarization_process(self):       
        # Ask for the output folder
        labeling_output_folder = filedialog.askdirectory(title="Select the output folder for standardized images")

        # Check if an input folder has been selected
        if self.labeling_image_paths:
            # Process each image in the input folder
            for image_path in self.labeling_image_paths:
                # Load the image
                image = cv2.imread(image_path)

                # Apply circular crop and resize
                dim = (min(image.shape[1], image.shape[0]), min(image.shape[1], image.shape[0]))
                circular = self.circle_crop(image)
                circular = cv2.cvtColor(circular, cv2.COLOR_BGR2RGB)
                labeling_processed_image = cv2.resize(circular, dim, interpolation=cv2.INTER_AREA)

                # Save the processed image
                filename = os.path.splitext(os.path.basename(image_path))[0] + ".jpg"
                output_path = os.path.join(labeling_output_folder, filename)
                cv2.imwrite(output_path, labeling_processed_image, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

            # Show a message box when the process is completed
            messagebox.showinfo("Process Completed", "All images have been processed and saved to the output folder.")
        else:
            messagebox.showwarning("Empty folder", "No input folder has been selected.")
            
    def circle_crop(self, image):   
        """
        Circular cut around the center of the image
        """    
        image                = self.crop_image_from_gray(image)    
        image                = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width, depth = image.shape    
        x                    = int(width/2)
        y                    = int(height/2)
        r                    = np.amin((x,y))
        circle_img           = np.zeros((height, width), np.uint8)
        cv2.circle(circle_img, (x,y), int(r), 1, thickness = -1)
        image                = cv2.bitwise_and(image, image, mask = circle_img)
        image                = self.crop_image_from_gray(image)
        return image  
    
    def crop_image_from_gray(self, image,tol=7):
        """
        Image cropping to avoid noise outside the retina. Image in BN
        """    
        if image.ndim == 2:
            mask = image > tol
            return image[np.ix_(mask.any(1),mask.any(0))]
        elif image.ndim == 3:
            gray_img    = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            mask        = gray_img > tol
            check_shape = image[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
            if (check_shape == 0): # La imagen fue muy oscura, y se recortó todo
                return image # Retornamos la imagen original
            else:
                img1 = image[:,:,0][np.ix_(mask.any(1),mask.any(0))]
                img2 = image[:,:,1][np.ix_(mask.any(1),mask.any(0))]
                img3 = image[:,:,2][np.ix_(mask.any(1),mask.any(0))]
                image = np.stack([img1,img2,img3],axis=-1)
            return image        
        
    def open_labelme(self):
        if self.labeling_image_paths:
            labelme_process = subprocess.Popen(["labelme", self.labeling_image_paths[self.labeling_current_image_index]])
            labelme_process.wait()
            self.load_image_to_label(self.labeling_image_paths[self.labeling_current_image_index])
        else:
            messagebox.showerror("Error", "No images available")
        self.labeling_update_buttons()    
    
    # labeling_previous_button_frame, labeling_next_button_frame, labeling_canvas
    # Methods: load_image_to_label, labeling_load_default_image, labeling_update_buttons

    def labeling_load_previous_image(self):
        if len(self.labeling_image_paths) > 0:
            self.labeling_current_image_index = (self.labeling_current_image_index - 1) % len(self.labeling_image_paths)
            labeling_image_path = self.labeling_image_paths[self.labeling_current_image_index]
            self.load_image_to_label(labeling_image_path)
        else:
            # Si no hay más imágenes, cargar la imagen por defecto "Caratula.jpg"
            self.labeling_current_image_index = 0
            self.labeling_load_default_image()
            print("No images available in the input folder")
        self.labeling_update_buttons()
        
    def labeling_load_next_image(self):
        if len(self.labeling_image_paths) > 0:
            self.labeling_current_image_index = (self.labeling_current_image_index + 1) % len(self.labeling_image_paths)
            labeling_image_path = self.labeling_image_paths[self.labeling_current_image_index]
            self.load_image_to_label(labeling_image_path)
        else:
            # Si no hay más imágenes, cargar la imagen por defecto "Caratula.jpg"
            self.labeling_current_image_index = 0
            self.labeling_load_default_image()
            print("No images available in the input folder")    
        self.labeling_update_buttons()        
        
    def labeling_update_image(self, event):
        # Update canvas dimensions
        self.labeling_canvas.config(width=event.width, height=event.height)

        # Check if there is a current image to display
        if hasattr(self, 'labeling_current_image_index') and hasattr(self, 'labeling_image_paths') and len(self.labeling_image_paths) > 0:
            # Reload the current image with the new canvas dimensions
            self.load_image_to_label(self.labeling_image_paths[self.labeling_current_image_index])
            # Check if there is label data for the current image
            if self.label_data:
                # Draw labels on canvas
                for shape in self.label_data:
                    label = shape["label"]
                    points = shape["points"]
                    self.labeling_canvas.create_polygon(points, fill="", outline="red", width=2)
        else:
            # If there is no current image, load the default image with the new canvas dimensions
            self.labeling_load_default_image()
        self.labeling_update_buttons()          
    
    # General
    # Methods: labeling_update_buttons
    
    def load_image_to_label(self, labeling_image_path):
        labeling_image = cv2.imread(labeling_image_path)
        labeling_image = cv2.cvtColor(labeling_image, cv2.COLOR_BGR2RGB)
        labeling_canvas_width = self.labeling_canvas.winfo_width()
        labeling_canvas_height = self.labeling_canvas.winfo_height()
        labeling_width_scale_factor = labeling_canvas_width / labeling_image.shape[1]
        labeling_height_scale_factor = labeling_canvas_height / labeling_image.shape[0]
        labeling_scale_factor = min(labeling_width_scale_factor, labeling_height_scale_factor)
        labeling_scaled_image = cv2.resize(labeling_image, (0, 0), fx=labeling_scale_factor, fy=labeling_scale_factor)
        labeling_pil_image = Image.fromarray(labeling_scaled_image)

        # Check for existing JSON files
        json_path =  labeling_image_path[:-4] + ".json"
        if os.path.exists(json_path):
            with open(json_path, "r") as f:
                label_data = json.load(f)

            # Extract label data from JSON
            shapes = label_data["shapes"]
            self.label_data = []
            for shape in shapes:
                label = shape["label"]
                points = shape["points"]
                self.label_data.append({"label": label, "points": points})
        else:
            self.label_data = []

        if self.label_data:
            labeling_pil_image = labeling_pil_image.convert("RGBA")
            overlay = Image.new("RGBA", labeling_pil_image.size, (255, 255, 255, 0))
            draw = ImageDraw.Draw(overlay)

            for shape in self.label_data:
                label = shape["label"]
                points = [(float(p[0]) * labeling_scale_factor, float(p[1]) * labeling_scale_factor) for p in shape["points"]]
                draw.polygon(points, outline="red", fill=(255, 0, 0, 30))  # Adjust the last value (0 to 255) for transparency

            labeling_pil_image = Image.alpha_composite(labeling_pil_image, overlay)

        self.labeling_image = ImageTk.PhotoImage(labeling_pil_image)
        self.labeling_canvas.delete("all")
        self.labeling_canvas.create_image(labeling_canvas_width/2, labeling_canvas_height/2, image=self.labeling_image)
        if len(self.labeling_image_paths) > 0:
            self.labeling_canvas.create_text(100, labeling_canvas_height - 30, text=f"Imagen {self.labeling_current_image_index+1} de {len(self.labeling_image_paths)}", fill="white", font=("Arial", 10))
    
        self.labeling_image_name_label.config(text=f"Imagen cargada: {os.path.basename(labeling_image_path)}")
        self.labeling_update_buttons()
        
    def labeling_update_buttons(self):
        labeling_num_images = len(self.labeling_image_paths)
        if labeling_num_images > 0:
            self.labeling_previous_button.config(state=tk.NORMAL)
            self.labeling_next_button.config(state=tk.NORMAL)
        else:
            self.labeling_previous_button.config(state=tk.DISABLED)
            self.labeling_next_button.config(state=tk.DISABLED)       
        
    def labeling_load_default_image(self):
        image = cv2.imread(self.default_image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        labeling_canvas_width = self.labeling_canvas.winfo_width()
        labeling_canvas_height = self.labeling_canvas.winfo_height()
        # Calculate the scale factors for width and height
        labeling_width_scale_factor = labeling_canvas_width / image.shape[1]
        labeling_height_scale_factor = labeling_canvas_height / image.shape[0]
        # Calculate the scale factor to make the image cover the largest possible area of the canvas without distorting its aspect ratio
        labeling_scale_factor = min(labeling_width_scale_factor, labeling_height_scale_factor)
        # Scale the image while maintaining its aspect ratio
        labeling_scaled_image = cv2.resize(image, (0, 0), fx=labeling_scale_factor, fy=labeling_scale_factor)
        labeling_pil_image = Image.fromarray(labeling_scaled_image)
        self.labeling_image = ImageTk.PhotoImage(labeling_pil_image)
        # Display the image on the canvas
        self.labeling_canvas.delete("all")
        self.labeling_canvas.create_image(labeling_canvas_width/2, labeling_canvas_height/2, image=self.labeling_image)  
        
################################################################################################################################


    #Step 3: Implement the pre training frame            
    
    def create_training_frame(self):
        self.training_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.training_frame, text="Pre - training Model - Yolo")

        # Widgets and logic for the training frame

        # Create a frame for input folder and pre-training methods.
        self.pre_training_frame = tk.Frame(self.training_frame)
        self.pre_training_frame.pack()
        
        # Create buttons for loading input, json to yolo conversion and standarization (dim)
        # Methods: select_input_folder
        
        # Create a button for input folder
        self.training_input_folder_button = tk.Button(self.pre_training_frame, text="Load folder to process for training", command=self.select_input_folder)
        self.training_input_folder_button.pack(side=tk.LEFT)  
        # Create a Radiobutton for json to yolo conversion
        json_yolo_button = ttk.Radiobutton(self.pre_training_frame, text="json to yolo", variable=self.preprocessing_method, value="json_yolo")
        json_yolo_button.pack(side=tk.RIGHT)
        # Create a Radiobutton for standarization (dim)
        standarization_button = ttk.Radiobutton(self.pre_training_frame, text="Standarization", variable=self.preprocessing_method, value="Standarization")
        standarization_button.pack(side=tk.RIGHT)        
        
        # Create a frame for input folder
        self.training_input_folder_frame = tk.Frame(self.training_frame)
        self.training_input_folder_frame.pack()
        # Create a no folder text
        self.training_input_folder_label = tk.Label(self.training_input_folder_frame, text="No folder available")
        self.training_input_folder_label.pack()        

        # Create buttons for enhancement techniques aplied to process folders
        # Methods: process_input_folder
        
        # Create a frame for label
        self.selection_label_frame = tk.Frame(self.training_frame)
        self.selection_label_frame.pack()
        # Create a enhancements text
        self.enhancements_label = ttk.Label(self.selection_label_frame, text="Select a color space:")
        self.enhancements_label.pack()   
        
        # Create a frame for enhancements label methods
        self.enhancements_label_frame = tk.Frame(self.training_frame)
        self.enhancements_label_frame.pack()        
                        
        # Create a Radiobutton for GRAY
        clahe_button = ttk.Radiobutton(self.enhancements_label_frame, text="GRAY", variable=self.preprocessing_method, value="GRAY")
        clahe_button.grid(row=0, column=0, sticky=tk.W)
        # Create a Radiobutton for HLS_FULL
        gamma_bn_button = ttk.Radiobutton(self.enhancements_label_frame, text="HLS_FULL", variable=self.preprocessing_method, value="HLS_FULL")
        gamma_bn_button.grid(row=0, column=1, sticky=tk.W)
        # Create a Radiobutton for HSV_FULL
        clahe_button = ttk.Radiobutton(self.enhancements_label_frame, text="HSV_FULL", variable=self.preprocessing_method, value="HSV_FULL")
        clahe_button.grid(row=0, column=2, sticky=tk.W)
        # Create a Radiobutton for LAB
        gamma_bn_button = ttk.Radiobutton(self.enhancements_label_frame, text="LAB", variable=self.preprocessing_method, value="LAB")
        gamma_bn_button.grid(row=0, column=3, sticky=tk.W)
        # Create a Radiobutton for LUV
        clahe_button = ttk.Radiobutton(self.enhancements_label_frame, text="LUV", variable=self.preprocessing_method, value="LUV")
        clahe_button.grid(row=1, column=0, sticky=tk.W)
        # Create a Radiobutton for XYZ
        gamma_bn_button = ttk.Radiobutton(self.enhancements_label_frame, text="XYZ", variable=self.preprocessing_method, value="XYZ")
        gamma_bn_button.grid(row=1, column=1, sticky=tk.W)
        # Create a Radiobutton for YCrCb
        clahe_button = ttk.Radiobutton(self.enhancements_label_frame, text="YCrCb", variable=self.preprocessing_method, value="YCrCb")
        clahe_button.grid(row=1, column=2, sticky=tk.W)
        # Create a Radiobutton for YUV
        gamma_bn_button = ttk.Radiobutton(self.enhancements_label_frame, text="YUV", variable=self.preprocessing_method, value="YUV")
        gamma_bn_button.grid(row=1, column=3, sticky=tk.W)               
        
        # Create a frame for preprocessing folders enhancement
        self.process_folder__frame = tk.Frame(self.training_frame)
        self.process_folder__frame.pack()
        # Create a button for processing  folder        
        process_folder_button = ttk.Button(self.process_folder__frame, text="Process folder", command=self.process_input_folder)
        process_folder_button.pack()

        self.method_label = ttk.Label(self.training_frame, text="")
        self.method_label.pack()


        
        # Crear un frame para el spliting
        self.split_label_frame = tk.Frame(self.training_frame)
        self.split_label_frame.pack()        
        
        self.split_percentages_label = ttk.Label(self.split_label_frame, text="Separation percentage (train, test, validation):")
        self.split_percentages_label.pack()
        
        input_folder_button = ttk.Button(self.split_label_frame, text="Select folder to divide", command=self.select_target_folder)
        input_folder_button.pack(side=tk.LEFT)

        # Crear la etiqueta "Training:"
        self.training_label = tk.Label(self.split_label_frame, text="Training size:")
        self.training_label.pack(side=tk.LEFT)
        
        self.train_percentage_entry = ttk.Entry(self.split_label_frame)
        self.train_percentage_entry.insert(0, "70")  # set a default value
        self.train_percentage_entry.pack(side=tk.LEFT)
        
        # Crear la etiqueta "Testing:"
        self.testing_label = tk.Label(self.split_label_frame, text="Testing size:")
        self.testing_label.pack(side=tk.LEFT)
        
        self.test_percentage_entry = ttk.Entry(self.split_label_frame)
        self.test_percentage_entry.insert(0, "20")  # set a default value
        self.test_percentage_entry.pack(side=tk.LEFT)
        
        # Crear la etiqueta "Validation:"
        self.validation_label = tk.Label(self.split_label_frame, text="Validation size:")
        self.validation_label.pack(side=tk.LEFT)
        
        self.validation_percentage_entry = ttk.Entry(self.split_label_frame)
        self.validation_percentage_entry.insert(0, "10")  # set a default value
        self.validation_percentage_entry.pack(side=tk.LEFT)

        # Create a frame for input folder
        self.spliting_input_folder_frame = tk.Frame(self.training_frame)
        self.spliting_input_folder_frame.pack()
        # Create a no folder text
        self.target_folder_label = tk.Label(self.spliting_input_folder_frame, text="No folder available")
        self.target_folder_label.pack()   
     
        
        split_button = ttk.Button(self.training_frame, text="Separate images", command=self.split_images)
        split_button.pack()
      

    
    
    
    
    
        # Add your widgets and logic for the training frame here
        
        # Crear elementos de entrada de valores para los parámetros de entrenamiento
        self.model_space_label = ttk.Label(self.training_frame, text="")
        self.model_space_label.pack()
        
        self.model_label = ttk.Label(self.training_frame, text="Training hyperparameters")
        self.model_label.pack()
        
        # Crear un frame para el batch size y su etiqueta
        self.batch_size_frame = tk.Frame(self.training_frame)
        self.batch_size_frame.pack()
        
        
        # Crear la etiqueta "batch size:"
        self.batch_size_label = tk.Label(self.batch_size_frame, text="batch size:")
        self.batch_size_label.pack(side=tk.LEFT)
        
        self.batch_size_entry = ttk.Entry(self.batch_size_frame)
        self.batch_size_entry.insert(0, "4")  # Valor por defecto
        self.batch_size_entry.pack()

        # Crear un frame para epochs y su etiqueta
        self.epochs_frame = tk.Frame(self.training_frame)
        self.epochs_frame.pack()
        
        # Crear la etiqueta "epochs:"
        self.epochs_label = tk.Label(self.epochs_frame, text="epochs:")
        self.epochs_label.pack(side=tk.LEFT)
        
        self.epochs_entry = ttk.Entry(self.epochs_frame)
        self.epochs_entry.insert(0, "100")  # Valor por defecto
        self.epochs_entry.pack()

        # Crear un frame para img size y su etiqueta
        self.img_frame = tk.Frame(self.training_frame)
        self.img_frame.pack()
        
        # Crear la etiqueta "img size:"
        self.img_size_label = tk.Label(self.img_frame, text="img size:")
        self.img_size_label.pack(side=tk.LEFT)
        
        self.img_size_training_entry = ttk.Entry(self.img_frame)
        self.img_size_training_entry.insert(0, "448 448")  # Valor por defecto
        self.img_size_training_entry.pack()

        self.data_yaml_button = ttk.Button(self.training_frame, text="Select data YAML file", command=self.select_data_yaml)
        self.data_yaml_button.pack()

        self.hyp_yaml_button = ttk.Button(self.training_frame, text="Select hyperparameters YAML file", command=self.select_hyp_yaml)
        self.hyp_yaml_button.pack()

        self.cfg_yaml_button = ttk.Button(self.training_frame, text="Select model configuration YAML file", command=self.select_cfg_yaml)
        self.cfg_yaml_button.pack()

        # Crear un frame para saved name y su etiqueta
        self.name_frame = tk.Frame(self.training_frame)
        self.name_frame.pack()
        
        # Crear la etiqueta "img size:"
        self.name_label = tk.Label(self.name_frame, text="File name:")
        self.name_label.pack(side=tk.LEFT)
        
        self.name_entry = ttk.Entry(self.name_frame)
        self.name_entry.insert(0, "yolo_train_")  # Valor por defecto
        self.name_entry.pack()

        self.weights_button = ttk.Button(self.training_frame, text="Select weight file", command=self.select_weights)
        self.weights_button.pack()

        self.train_button = ttk.Button(self.training_frame, text="Start training", command=self.start_training)
        self.train_button.pack()

        self.status_label = ttk.Label(self.training_frame, text="")
        self.status_label.pack()
        
    
    
    
    
    
    # Add the new methods to the MedicalGUI class

    # pre_training_frame
    # Methods:     

    def select_input_folder(self):
        self.preprocessing_input_folder = filedialog.askdirectory()
        if self.preprocessing_input_folder:
            self.training_input_folder_label.config(text=f"Carpeta de entrada seleccionada: {self.preprocessing_input_folder}")
            
    # enhancements_label_frame
    # Methods: gamma_correction, clahe, standarization_process, process_folder, labelme_to_yolo_mod, circle_crop, crop_image_from_gray
 
    def process_input_folder(self):
        if not self.preprocessing_method.get():
            messagebox.showerror("Error", "Please select a color space.")
            return
        if not self.preprocessing_input_folder:
            messagebox.showerror("Error", "Please select an input folder.")
            return

        method = self.preprocessing_method.get()
        output_folder = filedialog.askdirectory()

        if method == "Standarization":
            img_size = simpledialog.askinteger("img size value", "Enter a value for the img size (between 224 and 1792). 448 is recommended:", minvalue=224, maxvalue=1792)
            if img_size is None:
                return    
        elif method == "json_yolo":
            image_width = simpledialog.askinteger("img width value", "Enter a value for the img size (between 224 and 1792). 448 is recommended:", minvalue=224, maxvalue=1792)
            if image_width is None:
                return
            image_height = simpledialog.askinteger("img height value", "Enter a value for the img size (between 224 and 1792). 448 is recommended:", minvalue=224, maxvalue=1792)
            if image_height is None:
                return       
            label_map = {'RNFLd': 0}
                    
        processed_images = []
        for file_name in os.listdir(self.preprocessing_input_folder):
            if file_name.endswith(('.jpg', '.jpeg', '.png', '.tif', '.TIF', '.PNG', '.JPG', '.JPEG')):
                image_path = os.path.join(self.preprocessing_input_folder, file_name)
                image = cv2.imread(image_path)

                # apply the selected method
                if method == "Standarization":
                    processed_image = self.standarization_process(image, img_size)
                    processed_images.append(processed_image)
                    output_file_path = os.path.join(output_folder, os.path.splitext(file_name)[0] + '.jpg')
                    cv2.imwrite(output_file_path, processed_image,[int(cv2.IMWRITE_JPEG_QUALITY), 100])
                elif method == "json_yolo":
                    self.process_folder(self.preprocessing_input_folder, output_folder, image_width, image_height, label_map)                            
                elif method == "GRAY":
                    processed_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    processed_images.append(processed_image)
                    output_file_path = os.path.join(output_folder, file_name)
                    cv2.imwrite(output_file_path, processed_image)
                elif method == "HLS_FULL":
                    processed_image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS_FULL)
                    processed_images.append(processed_image)
                    output_file_path = os.path.join(output_folder, file_name)
                    cv2.imwrite(output_file_path, processed_image)
                elif method == "HSV_FULL":
                    processed_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV_FULL)
                    processed_images.append(processed_image)
                    output_file_path = os.path.join(output_folder, file_name)
                    cv2.imwrite(output_file_path, processed_image)
                elif method == "LAB":
                    processed_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
                    processed_images.append(processed_image)
                    output_file_path = os.path.join(output_folder, file_name)
                    cv2.imwrite(output_file_path, processed_image)
                elif method == "LUV":
                    processed_image = cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
                    processed_images.append(processed_image)
                    output_file_path = os.path.join(output_folder, file_name)
                    cv2.imwrite(output_file_path, processed_image)
                elif method == "XYZ":
                    processed_image = cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)
                    processed_images.append(processed_image)
                    output_file_path = os.path.join(output_folder, file_name)
                    cv2.imwrite(output_file_path, processed_image)
                elif method == "YCrCb":
                    processed_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
                    processed_images.append(processed_image)
                    output_file_path = os.path.join(output_folder, file_name)
                    cv2.imwrite(output_file_path, processed_image)
                elif method == "YUV":
                    processed_image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
                    processed_images.append(processed_image)
                    output_file_path = os.path.join(output_folder, file_name)
                    cv2.imwrite(output_file_path, processed_image)                                                            
                    
        messagebox.showinfo("Process completed", "All files in the output folder have been processed and saved.")
    
    def standarization_process(self, image, img_size):
        dim=()
        dim =(img_size,img_size)
        Circular   = self.circle_crop(image)                                     # Ajuste de la imagen a patron circular
        Circular   = cv2.cvtColor(Circular, cv2.COLOR_BGR2RGB)
        Escalada   = cv2.resize(Circular, dim, interpolation = cv2.INTER_AREA)              # Ajuste de la imagen a dimensiones dadas      
        return Escalada #cv2.cvtColor(Escalada, cv2.COLOR_BGR2RGB)
    
    def process_folder(self, input_folder, output_folder, image_width, image_height, label_map):
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        for filename in os.listdir(input_folder):
            if filename.endswith('.json'):
                input_json = os.path.join(input_folder, filename)
                output_txt = os.path.join(output_folder, filename.replace('.json', '.txt'))
                self.labelme_to_yolo_mod(input_json, output_txt, image_width, image_height, label_map)

    def circle_crop(self, image):   
        """
        Circular cut around the center of the image
        """    
        image                = self.crop_image_from_gray(image)    
        image                = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width, depth = image.shape    
        x                    = int(width/2)
        y                    = int(height/2)
        r                    = np.amin((x,y))
        circle_img           = np.zeros((height, width), np.uint8)
        cv2.circle(circle_img, (x,y), int(r), 1, thickness = -1)
        image                = cv2.bitwise_and(image, image, mask = circle_img)
        image                = self.crop_image_from_gray(image)
        return image                 

    def crop_image_from_gray(self, image,tol=7):
        """
        Image cropping to avoid noise outside the retina. Image in BN
        """    
        if image.ndim == 2:
            mask = image > tol
            return image[np.ix_(mask.any(1),mask.any(0))]
        elif image.ndim == 3:
            gray_img    = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            mask        = gray_img > tol
            check_shape = image[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
            if (check_shape == 0): # La imagen fue muy oscura, y se recortó todo
                return image # Retornamos la imagen original
            else:
                img1 = image[:,:,0][np.ix_(mask.any(1),mask.any(0))]
                img2 = image[:,:,1][np.ix_(mask.any(1),mask.any(0))]
                img3 = image[:,:,2][np.ix_(mask.any(1),mask.any(0))]
                image = np.stack([img1,img2,img3],axis=-1)
            return image    

    def labelme_to_yolo_mod(self, polygons_labelme_json, output_txt, image_width, image_height, label_map, new_width, new_height):
        with open(polygons_labelme_json, 'r') as f:
            labelme_data = json.load(f)

        shapes = labelme_data['shapes']
        yolo_polygons = []

        for shape in shapes:
            label = shape['label']
            class_id = label_map.get(label, -1)
            if class_id == -1:
                print(f"Warning: Label '{label}' not found in label_map, skipping shape")
                continue

            points = shape['points']
            yolo_polygon = [class_id]

            for point in points:
                x, y = point
                x_normalized = (x / image_width) * new_width
                y_normalized = (y / image_height) * new_height
                yolo_polygon.extend([x_normalized, y_normalized])

            yolo_polygons.append(yolo_polygon)

        with open(output_txt, 'w') as f:
            for polygon in yolo_polygons:
                polygon_line = ' '.join(str(coord) for coord in polygon)
                f.write(polygon_line + '\n')




                
                
                
                
                
                
                
    def create_split_frame(self):
        self.split_frame = ttk.Frame(self.notebook)
        self.notebook.add(self.split_frame, text="Split Dataset")

        # Add your widgets and logic for the split frame here
        
        input_folder_button = ttk.Button(self.split_frame, text="Seleccionar carpeta objetivo", command=self.select_target_folder)
        input_folder_button.pack()

        self.target_folder_label = ttk.Label(self.split_frame, text="")
        self.target_folder_label.pack()

        self.split_percentages_label = ttk.Label(self.split_frame, text="Porcentaje de separación (train, test, validation):")
        self.split_percentages_label.pack()

        # Crear un frame para el Training y su etiqueta
        self.training_frame = tk.Frame(self.split_frame)
        self.training_frame.pack()
        
        # Crear la etiqueta "Training:"
        self.training_label = tk.Label(self.training_frame, text="Training size:")
        self.training_label.pack(side=tk.LEFT)
        
        self.train_percentage_entry = ttk.Entry(self.training_frame)
        self.train_percentage_entry.insert(0, "70")  # set a default value
        self.train_percentage_entry.pack()

        # Crear un frame para el Testing y su etiqueta
        self.testing_frame = tk.Frame(self.split_frame)
        self.testing_frame.pack()
        
        # Crear la etiqueta "Testing:"
        self.testing_label = tk.Label(self.testing_frame, text="Testing size:")
        self.testing_label.pack(side=tk.LEFT)
        
        self.test_percentage_entry = ttk.Entry(self.testing_frame)
        self.test_percentage_entry.insert(0, "20")  # set a default value
        self.test_percentage_entry.pack()

        # Crear un frame para el Validation y su etiqueta
        self.validation_frame = tk.Frame(self.split_frame)
        self.validation_frame.pack()
        
        # Crear la etiqueta "Validation:"
        self.validation_label = tk.Label(self.validation_frame, text="Validation size:")
        self.validation_label.pack(side=tk.LEFT)
        
        self.validation_percentage_entry = ttk.Entry(self.validation_frame)
        self.validation_percentage_entry.insert(0, "10")  # set a default value
        self.validation_percentage_entry.pack()

        split_button = ttk.Button(self.split_frame, text="Separar imágenes", command=self.split_images)
        split_button.pack()

        self.split_status_label = ttk.Label(self.split_frame, text="")
        self.split_status_label.pack()

    # Add the new methods to the MedicalGUI class
    def select_target_folder(self):
        target_folder = filedialog.askdirectory()
        self.target_folder = target_folder
        self.target_folder_label.config(text="Carpeta objetivo seleccionada: " + self.target_folder)
    
    def is_number(self, value):
        try:
            float(value)
            return True
        except ValueError:
            return False

    def create_folder_structure(self, root_folder):
        folders = ['train', 'test', 'val']
        for folder in folders:
            os.makedirs(os.path.join(root_folder, folder, 'images'))
            os.makedirs(os.path.join(root_folder, folder, 'labels'))

    def split_images(self):
        if not self.target_folder:
            messagebox.showerror("Error", "Por favor, seleccione una carpeta objetivo.")
            return

        train_percentage_value = self.train_percentage_entry.get()
        test_percentage_value = self.test_percentage_entry.get()
        validation_percentage_value = self.validation_percentage_entry.get()

        if not self.is_number(train_percentage_value) or not self.is_number(test_percentage_value) or not self.is_number(validation_percentage_value):
            messagebox.showerror("Error", "Los porcentajes deben ser números.")
            return

        train_percentage = float(train_percentage_value)
        test_percentage = float(test_percentage_value)
        validation_percentage = float(validation_percentage_value)

        if train_percentage + test_percentage + validation_percentage != 100:
            messagebox.showerror("Error", "La suma de los porcentajes debe ser igual a 100.")
            return

        output_folder = filedialog.askdirectory(title="Seleccionar carpeta de salida")
        self.create_folder_structure(output_folder)
        

        image_files = [f for f in os.listdir(self.target_folder) if f.endswith(('.jpg', '.jpeg', '.png', '.tif', '.TIF', '.PNG', '.JPG', '.JPEG'))]
        random.shuffle(image_files)

        num_images = len(image_files)
        train_count = int(num_images * (train_percentage / 100))
        test_count = int(num_images * (test_percentage / 100))

        train_files = image_files[:train_count]
        test_files = image_files[train_count:train_count + test_count]
        val_files = image_files[train_count + test_count:]

        for file_type, file_list in zip(['train', 'test', 'val'], [train_files, test_files, val_files]):
            for file in file_list:
                src = os.path.join(self.target_folder, file)
                dst = os.path.join(output_folder, file_type, 'images', file)
                shutil.copy(src, dst)

                json_file = os.path.join(self.target_folder, os.path.splitext(file)[0] + ".json")
                yolo_file = os.path.join(self.target_folder, os.path.splitext(file)[0] + ".txt")

                if os.path.exists(json_file):
                    json_dst = os.path.join(output_folder, file_type, 'labels', os.path.splitext(file)[0] + ".json")
                    shutil.copy(json_file, json_dst)
                if os.path.exists(yolo_file):
                    yolo_dst = os.path.join(output_folder, file_type, 'labels', os.path.splitext(file)[0] + ".txt")
                    shutil.copy(yolo_file, yolo_dst)

        messagebox.showinfo("Separación completada", "Las imágenes se dividieron y copiaron en las carpetas de salida correspondientes.")

    def select_data_yaml(self):
        self.data_yaml_path = filedialog.askopenfilename(title="Seleccionar archivo YAML de data")

    def select_hyp_yaml(self):
        self.hyp_yaml_path = filedialog.askopenfilename(title="Seleccionar archivo YAML de hyperparámetros")

    def select_cfg_yaml(self):
        self.cfg_yaml_path = filedialog.askopenfilename(title="Seleccionar archivo YAML de configuración del modelo")

    def select_weights(self):
        self.weights_path = filedialog.askopenfilename(title="Seleccionar archivo de pesos")

    def start_training(self):
        batch_size = self.batch_size_entry.get()
        epochs = self.epochs_entry.get()
        img_size_training = self.img_size_training_entry.get()
        name = self.name_entry.get()

        # Validar valores de entrada
        if not batch_size.isdigit() or not epochs.isdigit() or not img_size_training.replace(" ","").isdigit():
            self.status_label.config(text="Error: los valores de entrada deben ser números.")
            return

        img_size_training = [int(x) for x in img_size_training.split()]
        if not self.data_yaml_path or not self.hyp_yaml_path or not self.cfg_yaml_path or not self.weights_path:
            self.status_label.config(text="Error: debe seleccionar todos los archivos necesarios para el entrenamiento.")
            return
        # Construir comando de entrenamiento
        train_command = f"python train.py --workers 1 --device 0 --batch-size {batch_size} --epochs {epochs} --img {img_size_training[0]} {img_size_training[1]} --data {self.data_yaml_path} --hyp {self.hyp_yaml_path} --cfg {self.cfg_yaml_path} --name {name} --weights {self.weights_path}"
        print(train_command)
    
        # Iniciar entrenamiento en una nueva ventana de terminal
        with open("YOLO_GUI.txt", "w") as output_file:
            subprocess.Popen(train_command, shell=True, stdout=output_file, stderr=subprocess.STDOUT)

        self.status_label.config(text="El entrenamiento se está ejecutando en una ventana de terminal separada.")
        
################################################################################################################################

    #Step 4: Implement the training frame            
    
    
    
root = tk.Tk()
root.update()
app = MedicalGUI(root)
root.mainloop()